<a href="https://colab.research.google.com/github/alixa2003/AI-ML-Internship-Tasks-Month2/blob/main/DHC_Internship_Part2_Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Task 1: News Topic Classifier Using BERT**

**Objective**:
Fine-tune a transformer model (e.g., BERT) to classify news headlines into topic categories.

**Dataset**:
AG News Dataset (Available on Hugging Face Datasets)

##**Importing Libraries**

In [ ]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score
import torch
import numpy as np
import gradio as gr

##**Loading Dataset**

In [ ]:
# Load the AG News dataset
dataset = load_dataset("ag_news")
print(dataset['train'][0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}


##**Defining Tokenizer**

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=64)

train_data = dataset['train'].map(tokenize, batched=True)
test_data = dataset['test'].map(tokenize, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##**Defining Training Arguments**

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    learning_rate=2e-5,
)


##**Defining Metrics**

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {'accuracy': acc, 'f1': f1}

##**Training of model**

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics
)

trainer.train()


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 1


wandb: You chose 'Create a W&B account'
wandb: Create an account here: https://wandb.ai/authorize?signup=true&ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: adarr417 (adarr417-university-of-management-technology-sialkot-campus) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, mcp] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.199100,0.183290,0.944474,0.944477
2,0.133900,0.197309,0.947368,0.947400
3,0.080400,0.231947,0.947632,0.947671


TrainOutput(global_step=22500, training_loss=0.15271093258327909, metrics={'train_runtime': 4735.7647, 'train_samples_per_second': 76.017, 'train_steps_per_second': 4.751, 'total_flos': 1.184021010432e+16, 'train_loss': 0.15271093258327909, 'epoch': 3.0})

##**Saving the model**

In [ ]:
trainer.save_model("./final_bert_agnews")
tokenizer.save_pretrained("./final_bert_agnews")

('./final_bert_agnews/tokenizer_config.json',
 './final_bert_agnews/special_tokens_map.json',
 './final_bert_agnews/vocab.txt',
 './final_bert_agnews/added_tokens.json')

##**Evaluation of Model**

In [ ]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.23194684088230133, 'eval_accuracy': 0.9476315789473684, 'eval_f1': 0.9476706911368392, 'eval_runtime': 29.7041, 'eval_samples_per_second': 255.857, 'eval_steps_per_second': 15.991, 'epoch': 3.0}


##**Classifying The Headlines**

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('./results/checkpoint-22500')

categories = ["World", "Sports", "Business", "Sci/Tech"]

def classify_headline(headline):
    inputs = tokenizer(headline, return_tensors="pt", truncation=True, padding=True, max_length=64)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return categories[prediction]

**Built The Gradio App**

In [ ]:
iface = gr.Interface(
    fn=classify_headline,           # Function to call
    inputs=gr.Textbox(lines=2, placeholder="Enter a news headline..."),
    outputs="text",                 # Display output as text
    title="News Topic Classifier",
    description="Classify news headlines into World, Sports, Business, or Sci/Tech."
)

iface.launch(share=True)  # 'share=True' gives a public URL

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e381518a83cc0e36de.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


##**Testing the Fine-Tuned Model**

In [ ]:
import torch.nn.functional as F

def predict_headline(headline):
    inputs = tokenizer(headline, return_tensors="pt", truncation=True, padding=True, max_length=64)
    outputs = model(**inputs)

    probs = F.softmax(outputs.logits, dim=1)[0]
    top_class = torch.argmax(probs).item()


    prob_dict = {categories[i]: float(probs[i].detach()) for i in range(len(categories))}

    return {
        "Predicted Class": categories[top_class],
        "Class Probabilities": prob_dict
    }

test_headlines = [
    "Pakistan defeats India in thrilling T20 World Cup match",
    "NASA launches new satellite to study climate change",
    "Stock markets drop after inflation fears",
    "Olympics 2028 to feature new sports categories",
    "Apple unveils next-generation iPhone with AI features"
]

for headline in test_headlines:
    result = predict_headline(headline)
    print(f"Headline: {headline}")
    print(f"Predicted Class: {result['Predicted Class']}")
    print(f"Class Probabilities: {result['Class Probabilities']}")
    print("="*70)

Headline: Pakistan defeats India in thrilling T20 World Cup match
Predicted Class: World
Class Probabilities: {'World': 0.997255265712738, 'Sports': 0.002519453875720501, 'Business': 0.00017337605822831392, 'Sci/Tech': 5.191682794247754e-05}
Headline: NASA launches new satellite to study climate change
Predicted Class: Sci/Tech
Class Probabilities: {'World': 0.02164989896118641, 'Sports': 0.0001381992333335802, 'Business': 0.00024259276688098907, 'Sci/Tech': 0.9779692888259888}
Headline: Stock markets drop after inflation fears
Predicted Class: Business
Class Probabilities: {'World': 0.011401926167309284, 'Sports': 5.305157174007036e-05, 'Business': 0.9879998564720154, 'Sci/Tech': 0.0005452276091091335}
Headline: Olympics 2028 to feature new sports categories
Predicted Class: World
Class Probabilities: {'World': 0.9347274899482727, 'Sports': 0.00921283382922411, 'Business': 0.012338261120021343, 'Sci/Tech': 0.043721381574869156}
Headline: Apple unveils next-generation iPhone with AI fe

In [ ]:
sample_headline = test_headlines[0]
result_for_sample = predict_headline(sample_headline)
prob_dict = result_for_sample['Class Probabilities']

top2 = sorted(prob_dict.items(), key=lambda x: x[1], reverse=True)[:2]
print(f"Top 2 predictions: {top2}")

Top 2 predictions: [('World', 0.997255265712738), ('Sports', 0.002519453875720501)]
